In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
from math import sqrt
from tabulate import tabulate

In [2]:
np.set_printoptions(formatter={'float_kind':"{:.8f}".format})
pd.options.display.float_format = '{:.8f}'.format

# Data creation

In [3]:
def pretty_view(a):
    pret=pd.DataFrame(data=a, index=list(range(1, a.shape[0]+1)), columns=list(range(1, a.shape[1]+1)))
    display(pret)

matrices by Pakulina A. N.

In [4]:
a1=np.matrix('-401.52 200.16; 1200.96 -601.68')
a1=np.array(a1)
pretty_view(a1)

,1,2
1,-401.52000000,200.16000000
2,1200.96000000,-601.68000000


In [5]:
a2=np.matrix('-403.15 200.95; 1205.70 -604.10')
a2=np.array(a2)
pretty_view(a2)

,1,2
1,-403.15000000,200.95000000
2,1205.70000000,-604.10000000


Hilbert matrices of order 2, 3

In [6]:
h2=linalg.hilbert(2)
pretty_view(h2)

,1,2
1,1.00000000,0.50000000
2,0.50000000,0.33333333


In [7]:
h3=linalg.hilbert(3)
pretty_view(h3)

,1,2,3
1,1.00000000,0.50000000,0.33333333
2,0.50000000,0.33333333,0.25000000
3,0.33333333,0.25000000,0.20000000


Diagonally dominant matrix

In [8]:
diag_m=np.matrix('''6 -1 0 0 0;
                    -1 6 -1 0 0;
                    0 -1 6 -1 0;
                    0 0 -1 6 -1;
                    0 0 0 -1 6''')
diag_m=np.array(diag_m)
pretty_view(diag_m)

,1,2,3,4,5
1,6,-1,0,0,0
2,-1,6,-1,0,0
3,0,-1,6,-1,0
4,0,0,-1,6,-1
5,0,0,0,-1,6


# Counting coefs and methods realisation

In [9]:
def count_b(A, x):  # считаем b для последующих вычислений
    b = np.dot(A, x)
    return b

In [10]:
def count_coefs(A, b):  # считаем alpha и beta
    alpha = np.array(np.zeros((A.shape[0], A.shape[0])))
    beta = np.array(np.zeros(b.shape[0]))
    for i in range(A.shape[0]):
        for j in range(A.shape[0]):
            if i != j:
                alpha[i][j] = - A[i][j] / A[i][i]
                beta[i] = b[i] / A[i][i]
            else:
                alpha[i][i] = 0
    return alpha, beta

In [11]:
def simple_iteration_method(alpha, beta, x, eps):  # метод простой итерации
    n_iter=1
    err = eps + 1
    while err > eps:
        err = np.linalg.norm(np.dot(alpha, x) + beta - x)
        x = np.dot(alpha, x) + beta
        n_iter += 1
    x = np.dot(alpha, x) + beta
    return x, n_iter

In [12]:
def seidel_method(A, b, eps): # функция возвращает x  и кол-во итераций для сравнения
    n_iter = 0
    x = np.array(np.zeros((b.shape[0])))
    err = eps + 1
    while err > eps:
        x_n = x.copy()
        for i in range(A.shape[0]):
            x1 = sum(A[i][j] * x_n[j] for j in range(i))
            x2 = sum(A[i][j] * x[j] for j in range(i + 1, A.shape[0]))
            x_n[i] = (b[i] - x1 - x2)/A[i][i]
        err = np.linalg.norm(x_n - x)
        n_iter += 1
        x = x_n
    return x, n_iter

# Print results

In [13]:
def print_result(A):
    pretty_view(A)
    x_w = np.random.uniform(0, 100, size=A.shape[0])
    b = count_b(A, x_w)
    alpha, beta = count_coefs(A, b)
    table=[['Приближение', "n_iter м. простых итераций","n_iter м. Зейделя", 
                'x - x_sim', 'x - x_seid']]
    for eps in (1e-4, 1e-7, 1e-10, 1e-13):
        row=[]
        row.append(eps)
        x_sim, n_iter_1 = simple_iteration_method(alpha, beta, beta, eps)
        x_seid, n_iter_2 = seidel_method(A, b, eps)
        row.append(n_iter_1)
        row.append(n_iter_2)
        row.append(np.linalg.norm(x_w - x_sim))
        row.append(np.linalg.norm(x_w- x_seid))
        table.append(row)
    return print(tabulate(table, tablefmt='fancy_grid', numalign="right"))

In [16]:
list_of_matrixes=[a1, a2, h2, diag_m]

In [17]:
for matrix in list_of_matrixes:
    print_result(matrix)

,1,2
1,-401.52000000,200.16000000
2,1200.96000000,-601.68000000


╒═════════════╤════════════════════════════╤═══════════════════╤════════════════════════╤════════════════════════╕
│ Приближение │ n_iter м. простых итераций │ n_iter м. Зейделя │ x - x_sim              │ x - x_seid             │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 0.0001      │ 5663                       │ 1624              │ 7.868875420336846e-05  │ 0.019953217023340516   │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-07       │ 8431                       │ 3009              │ 7.886347970936201e-08  │ 1.9898673261481005e-05 │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-10       │ 11199                      │ 4393              │ 7.654086267596506e-11  │ 1.993938385168084e-08  │
├─────────────┼────────────────────────────┼───────────────────┼────────────────

,1,2
1,-403.15000000,200.95000000
2,1205.70000000,-604.10000000


╒═════════════╤════════════════════════════╤═══════════════════╤════════════════════════╤════════════════════════╕
│ Приближение │ n_iter м. простых итераций │ n_iter м. Зейделя │ x - x_sim              │ x - x_seid             │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 0.0001      │ 5239                       │ 1645              │ 0.0001484510563130723  │ 0.01917289180989704    │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-07       │ 7907                       │ 2979              │ 1.4874495539679205e-07 │ 1.921103868789505e-05  │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-10       │ 10575                      │ 4313              │ 1.47703150991286e-10   │ 1.9246727682406012e-08 │
├─────────────┼────────────────────────────┼───────────────────┼────────────────

,1,2
1,1.00000000,0.50000000
2,0.50000000,0.33333333


╒═════════════╤════════════════════════════╤═══════════════════╤═══════════════════════╤════════════════════════╕
│ Приближение │ n_iter м. простых итераций │ n_iter м. Зейделя │ x - x_sim             │ x - x_seid             │
├─────────────┼────────────────────────────┼───────────────────┼───────────────────────┼────────────────────────┤
│ 0.0001      │ 101                        │ 45                │ 3.874501200301462e-05 │ 0.00023462513472999615 │
├─────────────┼────────────────────────────┼───────────────────┼───────────────────────┼────────────────────────┤
│ 1e-07       │ 149                        │ 69                │ 3.887635460969505e-08 │ 2.3542080912929293e-07 │
├─────────────┼────────────────────────────┼───────────────────┼───────────────────────┼────────────────────────┤
│ 1e-10       │ 197                        │ 93                │ 3.897097284112373e-11 │ 2.362227857710748e-10  │
├─────────────┼────────────────────────────┼───────────────────┼───────────────────────┼

,1,2,3,4,5
1,6,-1,0,0,0
2,-1,6,-1,0,0
3,0,-1,6,-1,0
4,0,0,-1,6,-1
5,0,0,0,-1,6


╒═════════════╤════════════════════════════╤═══════════════════╤════════════════════════╤════════════════════════╕
│ Приближение │ n_iter м. простых итераций │ n_iter м. Зейделя │ x - x_sim              │ x - x_seid             │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 0.0001      │ 12                         │ 8                 │ 1.0514450570319884e-05 │ 6.211373377866592e-06  │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-07       │ 18                         │ 11                │ 6.084757265850861e-09  │ 3.6040805168533608e-09 │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-10       │ 24                         │ 14                │ 3.5305363662987126e-12 │ 2.089993095218002e-12  │
├─────────────┼────────────────────────────┼───────────────────┼────────────────